# Soils Revealed precalculations with `Zarrs`
## Purpose
In this notebooks we will performed zonal statistics for different vector and raster data.
## Setup
### Library import
**Setup software libraries** 

In [21]:
import os 

import s3fs
import numpy as np
import xarray as xr
import pandas as pd
import geopandas as gpd
import regionmask

**Reads key-value pairs from a `.env` file**

In [ ]:
from dotenv import load_dotenv
load_dotenv()

**Setup `Dask` client**

In [ ]:
from dask.distributed import Client, LocalCluster
#cluster = LocalCluster(n_workers=1, threads_per_worker=36)
client = Client()  # start distributed scheduler locally.  Launch dashboard
client

### Utils

In [2]:
def read_dataset_from_zarr(s3_path, group):
    # Initilize the S3 file system
    s3 = s3fs.S3FileSystem(key=os.getenv("S3_ACCESS_KEY_ID"), secret=os.getenv("S3_SECRET_ACCESS_KEY"))
    store = s3fs.S3Map(root=s3_path, s3=s3, check=False)
    # Read Zarr file
    ds = xr.open_zarr(store=store, group=group, consolidated=True)
    
    # Change coordinates names
    ds = ds.rename({'x': 'lon', 'y': 'lat'})
    
    # Change dimension name
    if group == 'concentration':
        ds = ds.rename({'depht': 'depth'})
    
    return ds

In [13]:
def prepare_vector_data_political_boundaries(iso=None, tolerance=None):
    if iso:
        bboxs = pd.read_csv('../data/mbtiles/country_bbox.csv', converters={"bbox": literal_eval})
        bbox = bboxs[bboxs['gid_0'] == iso].bbox.iloc[0]
    else:
        bbox = None
        
    # Read Political boundaries:
    print('Reading Political boundaries')
    gdf_pb = gpd.read_file('../data/mbtiles/political_boundaries/political_boundaries.shp', bbox = bbox)
    # Select up to level 1 admin areas
    gdf_pb = gdf_pb[gdf_pb['level'] <= 1]
    #Simplify geometries
    if tolerance:
        gdf_pb['geometry'] = gdf_pb['geometry'].apply(lambda x: x.simplify(tolerance)) 
    # Add area in ha
    gdf_pb['area_ha'] = gdf_pb['geometry'].to_crs({'init': 'epsg:6933'}).map(lambda p: p.area / 10**4)    
    gdf_pb = gdf_pb[['name_0', 'gid_0', 'name_1', 'gid_1', 'level', 'bbox', 'area_ha', 'id', 'id_0', 'geometry']]
    

    vector_data = {'political_boundaries': gdf_pb}
    
    if iso:
        print('Intersecting areas with the selected country')
        gdf_pb = gdf_pb[gdf_pb['gid_0'] == iso]
            
        vector_data['political_boundaries'] = gdf_pb
        
        country = gdf_pb[gdf_pb['level'] == 0]['geometry'].iloc[0].buffer(0)
        
            
    # Split DataFrames to avoid overlapping geometries
    split_df_by = ['level', 'level', 'level', 'level']
    print("splitting DataFrames")
    for n, data_name in enumerate(list(vector_data.keys())):
        if split_df_by[n]:
            df = vector_data[data_name].copy()
            del vector_data[data_name]
            categories = list(df[split_df_by[n]].unique())
            for category in categories:
                vector_data[data_name+'_'+str(category)] = df[df[split_df_by[n]] == category]    

    # Set index
    for data_name in list(vector_data.keys()):
        vector_data[data_name] = vector_data[data_name].reset_index(drop=True).reset_index()
   
    return vector_data

In [23]:
def set_lat_lon_attrs(ds):
    """ Set CF latitude and longitude attributes"""
    ds["lon"] = ds.lon.assign_attrs({
      'axis' : 'X',
       'long_name' : 'longitude',
        'standard_name' : 'longitude',
         'stored_direction' : 'increasing',
          'type' : 'double',
           'units' : 'degrees_east',
            'valid_max' : 360.0,
             'valid_min' : -180.0
             })
    ds["lat"] = ds.lat.assign_attrs({
      'axis' : 'Y',
       'long_name' : 'latitude',
        'standard_name' : 'latitude',
         'stored_direction' : 'increasing',
          'type' : 'double',
           'units' : 'degrees_north',
            'valid_max' : 90.0,
             'valid_min' : -90.0
             })
    return ds

In [17]:
def create_ds_mask(df, ds, name, lon_name='lon', lat_name='lat'):
    # Create index column
    if 'index' not in df:
        df.reset_index(drop=True).reset_index()
    
    # Get mean ds cell area (in degrees) 
    mean_y_size = np.diff(ds.lat.values).mean()
    #print(mean_y_size)
    mean_x_size = np.diff(ds.lat.values).mean()
    #print(mean_x_size)
    mean_area = mean_y_size * mean_x_size
    print(f"The mean ds cell area is {np.round(mean_area, 6)} deg.\n")
    
    # Clip gdf to bounding box of ds
    xmin = ds.lon.min().values.tolist()
    xmax = ds.lon.max().values.tolist()
    ymin = ds.lat.min().values.tolist()
    ymax = ds.lat.max().values.tolist()
    df = df.cx[xmin:xmax, ymin:ymax]
    
    
    # Add area of geoms to gdf
    df = df.assign(area = df.area)
    df = df.assign(area_is_gt_cell = df['area'] > mean_area)
    print(f"Clipped gdf to dataset bounds, giving {len(df['index'])} potential geometries, of which {df['area_is_gt_cell'].sum()} are large enough.\n")
    
    print("Geometries smaller than mean cell size:")
    print(df.loc[df['area_is_gt_cell'] == False, ['index']])
    print("\n")

    # Extract indexes and geoms that are large enough!
    id_ints = df.loc[df['area_is_gt_cell'] == True, 'index'].values
    geoms = df.loc[df['area_is_gt_cell'] == True, 'geometry'].values
    
    print(f'Number of indexes: {len(id_ints)}')
    print(f'Number of geoms: {len(geoms)}')

    # create mask object
    da_mask = regionmask.Regions(
      name = name,
      numbers = id_ints,
      outlines = geoms)\
      .mask(ds, lon_name=lon_name, lat_name=lat_name)\
      .rename(name)

    # get the ints actually written to mask
    id_ints_mask = da_mask.to_dataframe().dropna()[name].unique()
    id_ints_mask = np.sort(id_ints_mask).astype('int')
    
    print(f'Number of ints in mask: {len(id_ints_mask)}')
    
    # update da attributes
    da_mask.attrs['id_ints'] = id_ints_mask
    da_mask = set_lat_lon_attrs(da_mask)
    return da_mask

## Zonal statistics
### Read raster data
**Read rasters as `xarray.Dataset` from `Zarr` in Amazon S3 bucket**

In [9]:
s3_path = 's3://soils-revealed/global-dataset.zarr' #'s3://soils-revealed/experimental-dataset.zarr' 
group = 'historic'#'concentration' #stocks
ds = read_dataset_from_zarr(s3_path, group)
ds

<xarray.Dataset>
Dimensions:  (depth: 3, time: 2, lat: 2160, lon: 4320)
Coordinates:
  * depth    (depth) object '0-30' '0-100' '0-200'
  * time     (time) <U4 'NoLU' '2010'
  * lon      (lon) float64 -180.0 -179.9 -179.8 -179.7 ... 179.8 179.9 180.0
  * lat      (lat) float64 89.96 89.88 89.79 89.71 ... -89.79 -89.87 -89.96
Data variables:
    stocks   (depth, time, lat, lon) float64 dask.array<chunksize=(1, 1, 270, 540), meta=np.ndarray>
Attributes:
    AREA_OR_POINT:  Area
    crs:            +init=epsg:4326
    is_tiled:       0
    nodatavals:     [-32767.0]
    offsets:        [0.0]
    res:            [0.0833333, 0.0833333]
    scales:         [1.0]
    transform:      [0.0833333, 0.0, -180.0, 0.0, -0.0833333, 89.99999999999999]

### Read vector data

In [29]:
gdf_pb = gpd.read_file('../data/mbtiles/political_boundaries/political_boundaries.shp')

In [34]:
list(vector_data.keys())

['political_boundaries_1', 'political_boundaries_0']

In [35]:
vector_data['political_boundaries_0']

,index,name_0,gid_0,name_1,gid_1,level,bbox,area_ha,id,id_0,geometry
0,0,Wallis and Futuna,WLF,None,None,0,"[-178.1825, -14.3691, -176.1242, -13.1828]",1.143859e+04,4,4,"MULTIPOLYGON (((-176.12610 -13.28720, -176.125..."
1,1,United States,USA,None,None,0,"[-179.1506, 18.9099, 179.7734, 72.6875]",9.449888e+08,7,7,"MULTIPOLYGON (((179.74390 51.91860, 179.61160 ..."
2,2,Reunion,REU,None,None,0,"[55.2163, -21.3899, 55.8374, -20.8718]",2.269184e+05,25,25,"POLYGON ((55.58820 -21.37460, 55.29290 -21.230..."
3,3,Palau,PLW,None,None,0,"[131.1199, 2.9714, 134.7214, 8.0942]",4.402345e+04,36,36,"MULTIPOLYGON (((134.71920 8.09360, 134.72080 8..."
4,4,Guam,GUM,None,None,0,"[144.6179, 13.2343, 144.9569, 13.6543]",6.853707e+04,37,37,"MULTIPOLYGON (((144.71880 13.25050, 144.61800 ..."
...,...,...,...,...,...,...,...,...,...,...,...
249,249,Vatican City,VAT,None,None,0,"[12.4456, 41.9001, 12.4585, 41.9076]",4.377659e+01,3859,3859,"POLYGON ((12.45560 41.90760, 12.45790 41.90130..."
250,250,Clipperton Island,XCL,None,None,0,"[-109.2349, 10.2851, -109.1982, 10.3204]",7.581023e+02,3860,3860,"POLYGON ((-109.22540 10.32040, -109.19900 10.3..."
251,251,Montserrat,MSR,None,None,0,"[-62.2418, 16.6749, -62.1446, 16.8243]",6.941860e+03,3861,3861,"POLYGON ((-62.15010 16.71210, -62.24180 16.722..."
252,252,"Virgin Islands, U.S.",VIR,None,None,0,"[-65.0865, 17.6729, -64.5649, 18.4157]",2.859792e+04,3862,3862,"MULTIPOLYGON (((-64.82210 17.68650, -64.88100 ..."


In [16]:
vector_data = prepare_vector_data_political_boundaries(iso=None, tolerance=0.075)

Reading Political boundaries


/home/iker/anaconda3/envs/geopy11/lib/python3.11/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


splitting DataFrames


### Zonal statistics
**Create the data mask by rasterizing the vector data**

In [24]:
names = list(vector_data.keys())[1:2]
for name in names:
    print(f'Create the data mask for {name}:')
    da_mask = create_ds_mask(vector_data[name], ds, name, lon_name='lon', lat_name='lat')
    
    ds[name] = da_mask

Create the data mask for political_boundaries_0:
The mean ds cell area is 0.006944 deg.

Clipped gdf to dataset bounds, giving 254 potential geometries, of which 233 are large enough.

Geometries smaller than mean cell size:
     index
14      14
45      45
72      72
100    100
101    101
114    114
115    115
155    155
184    184
190    190
225    225
232    232
233    233
236    236
237    237
243    243
245    245
246    246
249    249
250    250
251    251


Number of indexes: 233
Number of geoms: 233


/tmp/ipykernel_35759/1213896641.py:23: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  df = df.assign(area = df.area)


Number of ints in mask: 232


In [25]:
ds

<xarray.Dataset>
Dimensions:                 (depth: 3, time: 2, lat: 2160, lon: 4320)
Coordinates:
  * depth                   (depth) object '0-30' '0-100' '0-200'
  * time                    (time) <U4 'NoLU' '2010'
  * lon                     (lon) float64 -180.0 -179.9 -179.8 ... 179.9 180.0
  * lat                     (lat) float64 89.96 89.88 89.79 ... -89.87 -89.96
Data variables:
    stocks                  (depth, time, lat, lon) float64 dask.array<chunksize=(1, 1, 270, 540), meta=np.ndarray>
    political_boundaries_0  (lat, lon) float64 nan nan nan ... 147.0 147.0 147.0
Attributes:
    AREA_OR_POINT:  Area
    crs:            +init=epsg:4326
    is_tiled:       0
    nodatavals:     [-32767.0]
    offsets:        [0.0]
    res:            [0.0833333, 0.0833333]
    scales:         [1.0]
    transform:      [0.0833333, 0.0, -180.0, 0.0, -0.0833333, 89.99999999999999]